# DSU Worker (Kaggle)

Run all: sets up GPU wheels and launches the watcher. Uses Kaggle's /kaggle/working as the DSU root.


In [ ]:
# GPU + pip cache bootstrap (Kaggle)
import os, sys, subprocess, pathlib, re, json, time
BASE = pathlib.Path('/kaggle/working/M4L-Demucs')
SITE = BASE / '.venv' / 'site-packages'
WHEELS = BASE / 'wheelhouse'
MODEL_CACHE = BASE / 'model-cache'
PIP_CACHE = BASE / 'pip-cache'
MARKER = BASE / '.env.json'
for p in [SITE, WHEELS, MODEL_CACHE, PIP_CACHE]: p.mkdir(parents=True, exist_ok=True)

os.environ['DSU_ROOT'] = str(BASE)
os.environ.setdefault('XDG_CACHE_HOME', str(MODEL_CACHE))
os.environ.setdefault('DEMUCS_CACHE', str(MODEL_CACHE))
os.environ['PIP_CACHE_DIR'] = str(PIP_CACHE)

if str(SITE) not in sys.path:
    sys.path.insert(0, str(SITE))

print('Kaggle bootstrap. BASE=', BASE)
print('PIP cache:', PIP_CACHE)
print(subprocess.getoutput('nvidia-smi'))

def detect_cuda_flavor():
    smi = subprocess.getoutput('nvidia-smi')
    m = re.search(r'CUDA Version:\s*([0-9.]+)', smi)
    if not m:
        raise RuntimeError('GPU not detected. Enable GPU for Kaggle Notebook and rerun.')
    ver = m.group(1)
    if ver.startswith('12.4'):
        return ['cu124','cu126','cu121','cu118']
    if ver.startswith('12.6'):
        return ['cu126','cu121','cu118']
    if ver.startswith('12.'):
        return ['cu126','cu121','cu118']
    if ver.startswith('11.8'):
        return ['cu118','cu121']
    return ['cu126','cu121','cu118']

TORCH_MAP = {
    'cu124': ('2.5.1','2.5.1','https://download.pytorch.org/whl/cu124'),
    'cu126': ('2.8.0','2.8.0','https://download.pytorch.org/whl/cu126'),
    'cu121': ('2.4.0','2.4.0','https://download.pytorch.org/whl/cu121'),
    'cu118': ('2.1.2','2.1.2','https://download.pytorch.org/whl/cu118'),
}
DEMUCS_VER = '4.0.1'

cur = {}
if MARKER.exists():
    try: cur = json.loads(MARKER.read_text() or '{}')
    except Exception: cur = {}

flavors = detect_cuda_flavor()
print('CUDA flavors preference:', flavors)
chosen = None
for fl in flavors:
    torch_ver, ta_ver, idx_url = TORCH_MAP[fl]
    want = { 'torch': f'{torch_ver}+{fl}', 'torchaudio': f'{ta_ver}+{fl}', 'demucs': DEMUCS_VER }
    need_refresh = (cur != want)
    if need_refresh:
        print(f'Downloading wheels for {fl} …')
        cmd = ['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
               f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
        if subprocess.run(cmd, text=True).returncode != 0:
            print(f'Wheel download failed for {fl}. Direct install …')
            direct = [sys.executable,'-m','pip','install','--no-warn-script-location','--extra-index-url',idx_url,
                      '--upgrade','--target',str(SITE), f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
            if subprocess.run(direct, text=True).returncode != 0:
                print('Direct install failed; trying next flavor.')
                continue
            subprocess.run(['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
                            f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy'], text=True)
        MARKER.write_text(json.dumps(want))
        chosen = (fl, torch_ver, ta_ver)
        break
    else:
        chosen = (fl, torch_ver, ta_ver)
        break

if not chosen:
    raise RuntimeError('Failed to setup torch/demucs for CUDA flavors on Kaggle.')

fl, torch_ver, ta_ver = chosen
print('Installing torch/torchaudio … (', fl, ')')
wheel_install = [sys.executable,'-m','pip','install','--no-warn-script-location','--no-index','--find-links',str(WHEELS),
     '--upgrade','--target',str(SITE),'torch','torchaudio']
rc = subprocess.run(wheel_install, text=True).returncode
if rc != 0:
    print('Wheelhouse missing torch/torchaudio; installing from index …')
    index_install = [sys.executable,'-m','pip','install','--no-warn-script-location','--extra-index-url',TORCH_MAP[fl][2],
        '--upgrade','--target',str(SITE), f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}']
    subprocess.run(index_install, text=True, check=True)

print('Installing demucs …')
demucs_install = [sys.executable,'-m','pip','install','--no-warn-script-location','--upgrade','--target',str(SITE), f'demucs=={DEMUCS_VER}','ffmpeg-python','numpy']
subprocess.run(demucs_install, text=True, check=True)

import torch, demucs
print('Boot OK. torch', torch.__version__, 'cuda avail', torch.cuda.is_available(), 'demucs', demucs.__version__)


In [ ]:
# Launch watcher (Kaggle)
import urllib.request, pathlib, os
BASE = pathlib.Path(os.environ.get('DSU_ROOT','/kaggle/working/M4L-Demucs'))
w = BASE/'colab_watcher.py'
urllib.request.urlretrieve('https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py', w)
print('Watcher saved to', w)
!python -u - <<'PY'
import os, subprocess
os.environ['DSU_ROOT'] = '/kaggle/working/M4L-Demucs'
os.environ['DSU_REMOTE_JOBS_URL'] = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/remote_jobs.jsonl'
print('DSU_ROOT=', os.environ['DSU_ROOT'])
print('DSU_REMOTE_JOBS_URL=', os.environ['DSU_REMOTE_JOBS_URL'])
subprocess.call(['python','/kaggle/working/M4L-Demucs/colab_watcher.py'])
PY
